# Knowledge Augmented AI Agent - IBM impl

Implementation steps:

Pre-requirements:
1. Business domain knowledge (ontology and operation) documentations
2. (Optional) some docs need to match InstructLab required format if fine tune needed.

Steps: 
1. (Optional) Fine tune Gen AI models using InstructLab if needed. 
2. Load related business knowledge and operations doc to knowledge and workflow graphs and vector DBs to enable RAG 
3. GenAI-1 to get steps to fulfill customer request - Chain of Thoughts — business operation planning and workflow
4. Feed each work flow step to API (mock agent calls to execute actions)
5. If retrieving a doc (or some information) through API, feed contents to GenAI-2 to understand and generate summary
6. Loop step 5 and 6 till all workflow steps from 4 are done

Notes:
1. InstructLab only support fine tune Granit currently. 
2. To customize ML model, consider complexicity, time and cost, we prefer (1) prompt engineering; (2) RAG; (3) fine tune; (4) re-train. So if prompt or RAG can meet business requirements, no fine tune needed. 
3. For this POC, we skip InstructLab.
4. For IBM RAG support, in production, we shall use watsonx Discovery. But for this POC, using chromadb

## Initial setup

In [4]:
!pip install --upgrade chromadb sentence-transformers ibm-watsonx-ai

     |████████████████████████████████| 1.3 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 7.9 MB/s eta 0:00:01


     |████████████████████████████████| 53 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 913 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 9.8 MB/s eta 0:00:01
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.11.0-py3-none-any.whl size=73661 sha256=fe12f64c2ab1e35e46230c562011d05dd956ea17ab376a4909a845669c0764cd
  Stored in directory: /Users/jingdongsun/Library/Caches/pip/wheels/d3/a5/34/52d2e378d3d0446989ebcd214af8bad55beecf905258852b9f


  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.11.0-py3-none-any.whl size=536350 sha256=a4f26faeece6d65e88e982cb0c6d4d42daf928d61d1de1e8e67f0b14a0ea8e09
  Stored in directory: /Users/jingdongsun/Library/Caches/pip/wheels/ed/a7/cd/32a2cf8bae11ba6fc56451b7c46608cfd9176d1da8c6f73fa1
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3transfer-2.11.0-py3-none-any.whl size=89486 sha256=7378e848eb7f86b657a89ea0eabc3ad92c9eb0be7bca19a03d2841c49737a7fe
  Stored in directory: /Users/jingdongsun/Library/Caches/pip/wheels/4a/3f/7c/551692188f14bdb095e715dd7f1061fec13b3766d45f3c3296
Successfully built ibm-cos-sdk ibm-cos-sdk-core ibm-cos-sdk-s3transfer


In [2]:
import os
import getpass

def get_credentials():
    return {
        "url" : "https://us-south.ml.cloud.ibm.com",
        "apikey" : getpass.getpass("Please enter your api key (hit enter): ")
    }

In [ ]:
# Model setup
from ibm_watsonx_ai.foundation_models import Model

model_id = "meta-llama/llama-3-70b-instruct"
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

# Run this notebook with watsonx.ai platform, so PROJECT_ID and SPACE_ID should be available.
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")

model = Model(
    model_id = model_id,
    params = parameters,
    credentials = get_credentials(),
    project_id = project_id,
    space_id = space_id
    )

/Users/jingdongsun/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Initial WML client API
from ibm_watsonx_ai.client import APIClient
from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')
wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id, space_id=space_id)

vector_index_id = "c9dcaa9c-8472-416e-b0b1-62a793a46341"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

In [ ]:
# Chromadb
import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)


In [ ]:
# Couple mock API calls
def mock_API_action_1():
    print("Called mock_API_action_1 API...")
    
def mock_API_action_2():
    print("Called mock_API_action_2 API...")
    
mock_API_action_1()
mock_API_action_2()

## Setup knowledge and operation workflow graphs

Seamlessly integrate knowledge and workflow graphs:

Patent draft: https://ibm.anaqua.com/anaqua/Survey/Survey.aspx?SurveyAnswerGroupId=99421980

### Load knowledge and workflow into knowledge graph

This notebook uses simple HR related synthetic knowledge and workflow for demo.
For production, we shall be able to load business domain knowledge and workflow from business documentation (structured or unstructured) by automation tools or ML technology.

In [34]:
!pip install --upgrade owlready2
!pip install --upgrade rdflib

In [81]:
from owlready2 import *

owl_path = "file:///Users/jingdongsun/WatsonOrders/WO-knowledge-graph/ontology/ontology.owl"
onto = get_ontology(owl_path).load()
print("Loaded Watson Orders ontology: " + onto.base_iri) 

with onto:
    
    # ontology entity classes
    class Employee(Thing):
        pass
    class HR_Document(Thing):
        pass
    # workflow entity classes
    class Task(Thing):
        pass
    class OrchestrateSkill(Thing):
        pass
    class orchestrate_skill(Task >> OrchestrateSkill):
        python_name = "orchestrate_skill"
    class Rpc(Thing):
        pass
    class rpc(Task >> Rpc):
        python_name = "rpc"
    class Condition(Thing):
        pass
    
    # Entity properties
    class my_name(DataProperty):
        domain = [Employee, HR_Document, Task, Condition, OrchestrateSkill]
        range = [str]
    class address(DataProperty):
        domain = [Employee]
        range = [str]
    class email(DataProperty):
        domain = [Employee]
        range = [str]
    class role(DataProperty):
        domain = [Employee]
        range = [str]
    class is_manager(DataProperty):
        domain = [Employee]
        range = [bool]
    class labels(DataProperty):
        domain = [Task]
        range = [str]
    class include_docs(DataProperty):
        domain = [Task]
        range = [bool]
    class docs(DataProperty):
        domain = [Task]
        range = [str]
#    class orchestrate_skill(DataProperty):
#        domain = [Task]
#        range = [OrchestrateSkill]
#    class rpc(DataProperty):
#        domain = [Task]
#        range = [Rpc]
    class next_steps(DataProperty):
        domain = [Task, Condition]
        range = [str]
    class specs(DataProperty):
        domain = [OrchestrateSkill, Rpc]
        range = [str]
    class rpc_type(DataProperty):
        domain = [Rpc]
        range = [str]
    class url(DataProperty):
        domain = [Rpc]
        range = [str]
    
    # Object properties for relationships
    class has_manager(ObjectProperty):
        domain = [Employee]
        range = [Employee]
    class manager_of(ObjectProperty):
        domain = [Employee]
        range = [Employee]
        inverse_property = has_manager
    class has_owner(ObjectProperty):
        domain = [HR_Document, Task]
        range = [Employee]
    class owner_of(ObjectProperty):
        domain = [Employee]
        range = [HR_Document, Task]
        inverse_property = has_owner
    class has_reviewer(ObjectProperty):
        domain = [HR_Document]
        range = [Employee]
    class reviewer_of(ObjectProperty):
        domain = [Employee]
        range = [HR_Document]
        inverse_property = has_reviewer

print(Employee.iri, Employee.namespace)
print(has_manager.iri)
print(name.iri)
print("List all defined entities:")
print(list(onto.classes()))
print("List all defined properties:")
print(list(onto.properties()))

Loaded Watson Orders ontology: urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#
urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#Employee get_ontology("urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#")
urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#has_manager
urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#name
List all defined entities:
[webprotege.stanford.edu.RRifW0NkIELhl9bC98g8tK, webprotege.stanford.edu.Ri4chGJAR1X9UGn6WyYcHq, webprotege.stanford.edu.RKFWyKc1DFbLUEoygVvnOD, webprotege.stanford.edu.RiI4RKBkPsxOPprjstXCyD, webprotege.stanford.edu.R98KnW9FAyAIIFwsDnacJFk, webprotege.stanford.edu.R9E5XcZBCFHlf9aoQEUZJwo, webprotege.stanford.edu.RDKzB85eH37OYZOz3y2UR0W, webprotege.stanford.edu.RZafTHJfGt7cpdKt6cU1iK, webprotege.stanford.edu.RVYWHUFaCjfGzsb0nQGaFS, webprotege.stanford.edu.Rvf7KwZlH3edNqnO2CY4iq, ontology.Employee, ontology.HR_Document, ontology.Task, ontology.Condition, ontology.OrchestrateSkill, ontolo

In [82]:
# Parsing the json file to create knowledge (ontology and workflow) graph
import json 

# load json file
file_path = "/Users/jingdongsun/WatsonOrders/ontology-augmented-AI-agent/data/HR_Ops_v1.json"
with open(file_path, 'r') as f:
    entities = json.load(f)

# parse json file and create knowledge graph objects and relationships
# 1. Create all entities
for entity in entities:
    if (entity["entity_type"] == "Ontology:Employee"):
        my_employee = Employee()
        my_employee.my_name.append(entity["name"])
        my_employee.address.append(entity["address"])
        my_employee.email.append(entity["email"])
        my_employee.role.append(entity["role"])
        my_employee.is_manager.append(entity["is_manager"])
        print("Done create employee: ", my_employee, my_employee.my_name)
    elif (entity["entity_type"] == "Ontology:HR_Documentation"):
        my_doc = HR_Document()
        my_doc.my_name.append(entity["name"])
        print("Done create HR doc: ", my_doc, my_doc.my_name)
    elif (entity["entity_type"] == "Workflow:Task"):
        my_task = Task()
        my_task.my_name.append(entity["name"])
        
        if ("labels" in entity):
            my_task.labels.extend(entity["labels"])
        if ("include_docs" in entity):
            my_task.include_docs.append(entity["include_docs"])
        if ("docs" in entity):
            my_task.docs.extend(entity["docs"])
        if ("next_steps" in entity):
            my_task.next_steps.extend(entity["next_steps"])
            
        if ("orchestrate_skill" in entity):
            my_skill = OrchestrateSkill()
            my_skill.my_name.append(entity["orchestrate_skill"]["name"])
            my_skill.specs.append(entity["orchestrate_skill"]["specs"])
            print(my_task, my_task.my_name, my_task.orchestrate_skill, my_skill, my_skill.my_name, my_skill.specs)
            my_task.orchestrate_skill = [my_skill]

        if ("rpc" in entity.keys()):
            my_rpc = Rpc()
            my_rpc.rpc_type.append(entity["rpc"]["rpc_type"])
            my_rpc.url.append(entity["rpc"]["url"])
            my_rpc.specs.append(entity["rpc"]["specs"])
            print(my_task, my_task.my_name, my_task.rpc, my_rpc, my_rpc.rpc_type, my_rpc.url, my_rpc.specs)
            my_task.rpc = [my_rpc]
        
        print("Done create task: ", my_task, my_task.my_name)
    elif (entity["entity_type"] == "Workflow:Condition"):
        my_cond = Condition()
        my_cond.my_name.append(entity["name"])
        my_cond.next_steps.extend(entity["next_steps"])
        print("Done create condition: ", my_cond, my_cond.my_name)
    else:
        print("ERROR: entity_type " + entity["entity_type"] + " not right!!")
# 2. Set relationships
for entity in entities:
    if (entity["entity_type"] == "Ontology:Employee"):
        # resolve management relationships among employees
        if ("has_manager" in entity):
            for employee in Employee.instances():
                if (entity["has_manager"] in employee.my_name) :
                    my_manager = employee
                if (entity["name"] in employee.my_name) :
                    my_employee = employee
            my_manager.manager_of.append(my_employee)
    elif (entity["entity_type"] == "Ontology:HR_Documentation"):
        if ("owner" in entity):
            for employee in Employee.instances():
                if (entity["owner"] in employee.my_name) :
                    my_employee = employee
            for doc in HR_Document.instances():
                if (entity["name"] in doc.my_name) :
                    my_doc = doc
            my_doc.has_owner.append(my_employee)
        if ("reviewer" in entity):
            for doc in HR_Document.instances():
                if (entity["name"] in doc.my_name) :
                    my_doc = doc
            for reviewer in entity["reviewer"]:
                for employee in Employee.instances():
                    if (reviewer in employee.my_name) :
                        my_employee = employee
                my_doc.has_reviewer.append(my_employee)
    elif (entity["entity_type"] == "Workflow:Task"):
        if ("owner" in entity):
            for employee in Employee.instances():
                if (entity["owner"] in employee.my_name) :
                    my_employee = employee
            for task in Task.instances():
                if (entity["name"] in task.my_name) :
                    my_task = task
            my_task.has_owner.append(my_employee)

Done create employee:  ontology.employee136 ['John']
Done create employee:  ontology.employee137 ['Peter']
Done create employee:  ontology.employee138 ['Linda']
Done create HR doc:  ontology.hr_document45 ['Performance Doc']
ontology.task76 ['Submit Promption'] [] ontology.orchestrateskill45 ['My skill 1'] ['{json formated string}']
Done create task:  ontology.task76 ['Submit Promption']
ontology.task77 ['Review Promption'] [] ontology.rpc22 ['REST API'] ['abc.com/rest'] ['{json formated string}']
Done create task:  ontology.task77 ['Review Promption']
Done create condition:  ontology.condition10 ['Approval']
ontology.task78 ['Prompt Employee'] [] ontology.orchestrateskill46 ['my skill 2'] ['{json formated string}']
Done create task:  ontology.task78 ['Prompt Employee']
ontology.task79 ['Reject Request'] [] ontology.rpc23 ['REST API'] ['abc.com/rest'] ['{json formated string}']
Done create task:  ontology.task79 ['Reject Request']


### Knowledge and workflow queries

Current popular qy=uery language for knowledge graph is SPARQL. For example, 

To list all item which is food and have children active.

list(default_world.sparql("""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX myns: <urn:webprotege:ontology:70d996e9-fb1e-4b2b-ab25-d20cd441d73b#>

        SELECT DISTINCT ?prod
        WHERE {
            ?prod rdf:type myns:Item .
            ?prod myns:has_child ?item .
            ?item myns:status "ACTIVE" .
            ?prod myns:category ?category .
            FILTER (?category = "FOOD")
        }
    """))

References:
1. https://www.w3.org/TR/rdf-sparql-query/
2. https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial

Under investigation: Enhance SPARQL to support workflows.

## Ontology Augmented AI Agent

In [ ]:
# User prompt inputs

prompt_input = """<|start_header_id|>system<|end_header_id|>

You always answer the questions with markdown formatting. The markdown formatting you support: headings, bold, italic, links, tables, lists, code blocks, and blockquotes. You must omit that you answer the questions with markdown.

Any HTML tags must be wrapped in block quotes, for example ```<html>```. You will be penalized for not rendering code in block quotes.

When returning code blocks, specify language.

Given the document and the current conversation between a user and an assistant, your task is as follows: answer any user query by using information from the document. Always answer as helpfully as possible, while being safe. When the question cannot be answered using the context or document, output the following response: "I cannot answer that question based on the provided document.".

Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<|eot_id|><|start_header_id|>user<|end_header_id|>

__grounding__<|eot_id|><|start_header_id|>user<|end_header_id|>

can you give me a summary of all amazon ec2 features?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Amazon EC2 Features**
=====================

### **Instance Choice**

* Broadest and deepest instance choice to match workload needs
* General purpose, compute optimized, memory optimized, storage optimized, and accelerated computing instance types
* Processors from Intel, AMD, NVIDIA, and AWS for additional performance and cost optimizations
* Local storage and enhanced networking options available with instance types
* Bare metal instances for direct access to processor and memory

### **Global Infrastructure**

* Multiple locations with Regions and Availability Zones
* Availability Zones are distinct locations engineered to be insulated from failures in other Availability Zones
* Low latency network connectivity to other Availability Zones in the same Region
* Regions consist of one or more Availability Zones and are geographically dispersed
* 99.99% availability for each Amazon EC2 Region

### **Operating Systems and Software**

* Growing list of operating systems, including Microsoft Windows and Linux distributions (Amazon Linux 2, Ubuntu, Red Hat Enterprise Linux, CentOS, SUSE, and Debian)
* AWS Marketplace features a wide selection of commercial and free software from well-known vendors

### **High Performance Computing (HPC)**

* High Throughput and Low Latency with High Performance Computing (HPC) Clusters
* Cluster Compute, Cluster GPU, and High Memory Cluster instances for high-performance network capability
* Programmatically launch instances into clusters for low-latency network performance

### **Time Sync Service**

* Highly accurate, reliable, and available time source to AWS services, including EC2 instances

### **Maintenance**

* Routine hardware, software, power, and network maintenance to improve application uptime and reduce operation effort

### **Security**

* Access services hosted on AWS easily and securely with AWS PrivateLink
* Keep all network traffic within the AWS network with AWS PrivateLink

### **Other Features**

* AWS Compute Optimizer for recommendations on optimal AWS Compute resources for workloads to reduce costs and improve performance<|eot_id|><|start_header_id|>user<|end_header_id|>

what are its advantages?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Advantages of Amazon EC2**
==========================

### **Improved Application Uptime and Reduced Operational Effort**

* Routine maintenance with minimal disruption to instances
* Live update and live migration technologies for non-intrusive maintenance

### **Broadest and Deepest Instance Choice**

* Match workload needs with a wide range of instance types
* Optimize compute, memory, storage, and networking resources

### **High Performance and Low Latency**

* Enhanced Networking for high packet-per-second performance and low latency
* Elastic Fabric Adapter for high levels of inter-instance communications

### **Global Infrastructure**

* Multiple locations with Regions and Availability Zones for high availability and low latency
* 99.99% availability for each Amazon EC2 Region

### **Cost-Effective**

* Pay only for the resources used
* AWS Compute Optimizer for recommendations on optimal AWS Compute resources to reduce costs and improve performance

### **Security**

* Access services hosted on AWS easily and securely with AWS PrivateLink
* Keep all network traffic within the AWS network with AWS PrivateLink

### **Flexibility and Scalability**

* Scale up or down to match changing workload needs
* Launch instances in separate Availability Zones for high availability

### **Integration with Other AWS Services**

* Seamlessly integrate with other AWS services, such as Amazon S3, Amazon RDS, and Amazon DynamoDB"""


In [ ]:
# Feed user request to Gen AI-1 to generate CoT

# Sample RAG query
question = input("Question: ")
grounding = proximity_search(question)
formattedQuestion = f"""<|begin_of_text|><|eot_id|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
print(f"AI: {generated_response}")

In [ ]:
# parse CoT responses, loop into each workflow steps to execute